## 1. Import all the packages

In [27]:
%%time
import numpy as np
import math
import pandas as pd
import pickle, os, math
from keras.models import Sequential
from keras.callbacks import Callback
from keras.layers import Dense
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

Wall time: 0 ns


## 2. Import Data

Import the data from the data folder. This step only uses the data for surviving in the rain of topedoes for the fighter. There are two types of data.

1. with no fires from the fighter,
2. with fires from the fighter.

The difference lies in the number of data points in each class. With fires from the fighter, the number of possible movements are higher than the case without fires from the fighter. Also the total number of data points is less than the second case, because the lives are consumed faster. As can be seen, the first dataset has 154k data points, the second dataset has 200k data points.

In [28]:
%%time
data_dir = os.path.join(os.curdir, 'Data', '200000_sur_nofires', 'basic_data_pics.pkl')
with open(data_dir, 'rb') as in_file:
    ot = pickle.load(in_file)
data_pics_nofires = ot['data']
target_pics_nofires = ot['target']
print(data_pics_nofires.shape)

(154332, 24)
Wall time: 58 ms


In [29]:
%%time
data_dir = os.path.join(os.curdir, 'Data', '200000_sur_fires', 'basic_data_pics.pkl')
with open(data_dir, 'rb') as in_file:
    ot = pickle.load(in_file)
data_pics_fires = ot['data']
target_pics_fires = ot['target']
print(data_pics_fires.shape)

(200036, 24)
Wall time: 63 ms


## 3. Preprocess First Dataset (without fires)

The first data is unbalanced with classes. The ratio of classes 0 and 1 are almost 1:5. Very unbalanced.

In [31]:
X_train_nf, X_test_nf, y_train_nf, y_test_nf = train_test_split(
    data_pics_nofires, target_pics_nofires, test_size=0.1, random_state=152)
X_train.shape
X_train_train_nf, X_vali_nf, y_train_train_nf, y_vali_nf = train_test_split(
    X_train_nf, y_train_nf, test_size=0.3, random_state=15545)
print('class 0 has ' + str(len(y_train_nf.index[y_train_nf[0] == 0].tolist())) + ' points')
print('class 1 has ' + str(len(y_train_nf.index[y_train_nf[0] == 1].tolist())) + ' points')

class 0 has 23761 points
class 1 has 115137 points


In [32]:
index_0 = y_train_nf.index[y_train_nf[0] == 0].tolist()
index_1 = y_train_nf.index[y_train_nf[0] != 0].tolist()
index_1_comparable_to_0 = np.random.choice(index_1, math.floor(len(index_0) * 1))
samples_nf = np.concatenate([index_0, index_1_comparable_to_0])
print(str(len(index_0)) + ' + ' + str(len(index_1_comparable_to_0)) + ' = ' + str(len(samples_nf)))

23761 + 23761 = 47522


`small_data_nf` and `small_target_nf` are the balanced data. All variables with `_small` is the balanced result.

In [33]:
small_data_nf = data_pics_nofires.iloc[samples_nf, :]
small_target_nf = target_pics_nofires.iloc[samples_nf, :]

X_train_small_nf, X_test_small_nf, y_train_small_nf, y_test_small_nf = train_test_split(
    small_data_nf, small_target_nf, test_size=0.3, random_state=1152)
X_train_train_small_nf, X_vali_small_nf, y_train_train_small_nf, y_vali_small_nf = train_test_split(
    X_train_small_nf, y_train_small_nf, test_size=0.3, random_state=8155)

## 4. Preprocess Second Dataset (with fires)

The second data is unbalanced with classes. The ratio of classes 0 and 1 are almost 1:10. More unbalanced compared to the second dataset. It is because of the fires helps with the possible movements.

In [34]:
X_train_fs, X_test_fs, y_train_fs, y_test_fs = train_test_split(
    data_pics_fires, target_pics_fires, test_size=0.1, random_state=218)
X_train_fs.shape
X_train_train_fs, X_vali_fs, y_train_train_fs, y_vali_fs = train_test_split(
    X_train_fs, y_train_fs, test_size=0.3, random_state=1515615)
print('class 0 has ' + str(len(y_train_fs.index[y_train_fs[0] == 0].tolist())) + ' points')
print('class 1 has ' + str(len(y_train_fs.index[y_train_fs[0] == 1].tolist())) + ' points')

class 0 has 15101 points
class 1 has 164931 points


Create balanced classes sample data, each class has 15061 data points

In [35]:
index_0 = y_train_fs.index[y_train_fs[0] == 0].tolist()
index_1 = y_train_fs.index[y_train_fs[0] != 0].tolist()
index_1_comparable_to_0 = np.random.choice(index_1, math.floor(len(index_0) * 1))
samples_fs = np.concatenate([index_0, index_1_comparable_to_0])
print(str(len(index_0)) + ' + ' + str(len(index_1_comparable_to_0)) + ' = ' + str(len(samples_fs)))

15101 + 15101 = 30202


`small_data_fs` and `small_target_fs` are the balanced data. All variables with `_small` is the balanced result.

In [36]:
small_data_fs = data_pics_fires.iloc[samples_fs, :]
small_target_fs = target_pics_fires.iloc[samples_fs, :]

X_train_small_fs, X_test_small_fs, y_train_small_fs, y_test_small_fs = train_test_split(
    small_data_fs, small_target_fs, test_size=0.3, random_state=152)
X_train_train_small_fs, X_vali_small_fs, y_train_train_small_fs, y_vali_small_fs = train_test_split(
    X_train_small_fs, y_train_small_fs, test_size=0.3, random_state=152)

## 4. Pretraining Several Models

For the scikit models, target (y) has to be ravelled.

In [37]:
y_train_small_nf_m = np.ravel(y_train_small_nf)
y_test_small_nf_m = np.ravel(y_test_small_nf)
y_train_train_small_nf_m = np.ravel(y_train_train_small_nf)
y_vali_small_nf_m = np.ravel(y_vali_small_nf)
y_train_nf_m = np.ravel(y_train_nf)
y_test_nf_m = np.ravel(y_test_nf)
y_train_train_nf_m = np.ravel(y_train_train_nf)
y_vali_nf_m = np.ravel(y_vali_nf)

In [38]:
y_train_small_fs_m = np.ravel(y_train_small_fs)
y_test_small_fs_m = np.ravel(y_test_small_fs)
y_train_train_small_fs_m = np.ravel(y_train_train_small_fs)
y_vali_small_fs_m = np.ravel(y_vali_small_fs)
y_train_fs_m = np.ravel(y_train_fs)
y_test_fs_m = np.ravel(y_test_fs)
y_train_train_fs_m = np.ravel(y_train_train_fs)
y_vali_fs_m = np.ravel(y_vali_fs)

### 4.1 SVC with scikit-learn

1. **no fires**

In [39]:
clf_nf = SVC(C=10.0, gamma='auto', verbose=True)
clf_nf.fit(X_train_small_nf, y_train_small_nf_m)
clf_nf.score(X_test_small_nf, y_test_small_nf_m)

[LibSVM]

0.945991442800028

In [40]:
cross_val_score(clf_nf, X_train_small_nf, y_train_small_nf_m, cv=6)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

array([0.94048693, 0.94642857, 0.94678932, 0.94660895, 0.95147908,
       0.93921356])

2. ** With fires**

In [41]:
clf_fs = SVC(C=10.0, gamma='auto', verbose=True)
clf_fs.fit(X_train_small_fs, y_train_small_fs_m)
clf_fs.score(X_test_small_fs, y_test_small_fs_m)

[LibSVM]

0.9381966670345436

In [42]:
cross_val_score(clf_fs, X_train_small_fs, y_train_small_fs_m, cv=6)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

array([0.93728717, 0.93955732, 0.93586833, 0.93897247, 0.93698552,
       0.94010786])

The results of different datasets are almost the same

### 4.2 MLPC with scikit-learn

1. **no fires**

In [43]:
mlpc_nf = MLPClassifier(hidden_layer_sizes=(50, 20),
                        alpha=0.15, max_iter=1000, batch_size=5000,
                        verbose=False, learning_rate_init=0.01, tol=1e-5,
                        learning_rate='adaptive')

mlpc_nf.fit(X_train_small_nf, y_train_small_nf_m)
mlpc_nf.score(X_test_small_nf, y_test_small_nf_m)

0.7848074630006313

In [44]:
cross_val_score(mlpc_nf, X_train_small_nf, y_train_small_nf_m, cv=5)

array([0.87571386, 0.89493462, 0.78505937, 0.82684503, 0.86410102])

2. ** With fires**

In [46]:
mlpc_fs = MLPClassifier(hidden_layer_sizes=(50, 20),
                        alpha=0.15, max_iter=1000, batch_size=5000,
                        verbose=False, learning_rate_init=0.01, tol=1e-5,
                        learning_rate='adaptive')

mlpc_1.fit(X_train_small_fs, y_train_small_fs_m)
mlpc_1.score(X_test_small_fs, y_test_small_fs_m)

0.887098554243461

In [47]:
cross_val_score(mlpc_fs, X_train_small_fs, y_train_small_fs_m, cv=5)

array([0.88507921, 0.88129581, 0.80368969, 0.84531693, 0.84362432])

### 4.3 NN with Keras

1. **no fires**

In [51]:
model_NNK_nf = Sequential()
model_NNK_nf.add(Dense(units=12, activation='relu', input_dim=24))
model_NNK_nf.add(Dense(units=6, activation='relu'))
model_NNK_nf.add(Dense(units=1, activation='sigmoid'))
model_NNK_nf.compile(loss='mean_squared_error',
                     optimizer='rmsprop',
                     metrics=['accuracy'])

model_NNK_nf.fit(X_train_train_small_nf, y_train_train_small_nf,
                 validation_data=(X_vali_small_nf, y_vali_small_nf),
                 epochs=20, batch_size=4096, verbose=1)

Train on 23285 samples, validate on 9980 samples
Epoch 1/20
23285/23285 [==============================] - 0s 14us/step - loss: 0.2685 - acc: 0.5793 - val_loss: 0.2169 - val_acc: 0.6623
Epoch 2/20
23285/23285 [==============================] - 0s 2us/step - loss: 0.2083 - acc: 0.6774 - val_loss: 0.2003 - val_acc: 0.6974
Epoch 3/20
23285/23285 [==============================] - 0s 2us/step - loss: 0.1948 - acc: 0.7138 - val_loss: 0.1965 - val_acc: 0.7007
Epoch 4/20
23285/23285 [==============================] - 0s 2us/step - loss: 0.1910 - acc: 0.7191 - val_loss: 0.1922 - val_acc: 0.7134
Epoch 5/20
23285/23285 [==============================] - 0s 2us/step - loss: 0.1879 - acc: 0.7279 - val_loss: 0.1897 - val_acc: 0.7251
Epoch 6/20
23285/23285 [==============================] - 0s 2us/step - loss: 0.1827 - acc: 0.7394 - val_loss: 0.1857 - val_acc: 0.7204
Epoch 7/20
23285/23285 [==============================] - 0s 1us/step - loss: 0.1805 - acc: 0.7397 - val_loss: 0.1813 - val_acc: 0.732

In [52]:
loss_and_metrics = model_NNK_nf.evaluate(X_test_small_nf, y_test_small_nf, batch_size=128)
print(loss_and_metrics)
y_predict_nf = model_NNK_nf.predict(X_test_small_nf, batch_size=None, verbose=0)
print(np.sum(y_predict_nf) / len(y_predict_nf))

14257/14257 [==============================] - 0s 5us/step
[0.15824912719968376, 0.7735147647355309]
0.4994975398598057


2. ** With fires**

In [53]:
model_NNK_fs = Sequential()
model_NNK_fs.add(Dense(units=12, activation='relu', input_dim=24))
model_NNK_fs.add(Dense(units=6, activation='relu'))
model_NNK_fs.add(Dense(units=1, activation='sigmoid'))
model_NNK_fs.compile(loss='mean_squared_error',
                     optimizer='rmsprop',
                     metrics=['accuracy'])

model_NNK_fs.fit(X_train_train_small_fs, y_train_train_small_fs,
          validation_data=(X_vali_small_fs, y_vali_small_fs), 
          epochs=20, batch_size=4096, verbose=1)

Train on 14798 samples, validate on 6343 samples
Epoch 1/20
14798/14798 [==============================] - 0s 18us/step - loss: 0.5005 - acc: 0.4980 - val_loss: 0.4819 - val_acc: 0.5067
Epoch 2/20
14798/14798 [==============================] - 0s 2us/step - loss: 0.4496 - acc: 0.4978 - val_loss: 0.3501 - val_acc: 0.5059
Epoch 3/20
14798/14798 [==============================] - 0s 2us/step - loss: 0.3248 - acc: 0.4926 - val_loss: 0.2662 - val_acc: 0.4674
Epoch 4/20
14798/14798 [==============================] - 0s 2us/step - loss: 0.2471 - acc: 0.4574 - val_loss: 0.2083 - val_acc: 0.6852
Epoch 5/20
14798/14798 [==============================] - 0s 2us/step - loss: 0.2054 - acc: 0.6980 - val_loss: 0.1986 - val_acc: 0.7369
Epoch 6/20
14798/14798 [==============================] - 0s 2us/step - loss: 0.2002 - acc: 0.7494 - val_loss: 0.1956 - val_acc: 0.6826
Epoch 7/20
14798/14798 [==============================] - 0s 1us/step - loss: 0.1969 - acc: 0.7524 - val_loss: 0.1929 - val_acc: 0.749

In [54]:
loss_and_metrics = model_NNK_fs.evaluate(X_test_small_fs, y_test_small_fs, batch_size=128)
print(loss_and_metrics)
y_predict_fs = model_NNK_fs.predict(X_test_small_fs, batch_size=None, verbose=0)
print(np.sum(y_predict_fs) / len(y_predict_fs))

9061/9061 [==============================] - 0s 7us/step
[0.1744909577417842, 0.8367729828447422]
0.3570299997930692


## 5. Selecting models and Saving Models

the SVC model apparently has a higher accuracy, grid search is used to find the best parameters.

1. **no fires**

In [55]:
parameters = {'C': [0.1, 1, 10], 'kernel': ['rbf', 'linear']}
from sklearn.model_selection import GridSearchCV
clf_nf_2 = SVC(gamma='auto', verbose=True)
svc_vc_nf = GridSearchCV(clf_nf_2, parameters, cv=6, refit=True)
svc_vc_nf.fit(X_train_small_nf, y_train_small_nf_m)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

GridSearchCV(cv=6, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.1, 1, 10], 'kernel': ['rbf', 'linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [56]:
print(svc_vc_nf.cv_results_)

{'mean_fit_time': array([ 20.33870033,  20.1581823 ,  20.92692582,  47.23078652,
        37.32616735, 250.03938635]), 'std_fit_time': array([ 1.89203288,  0.94841152,  0.19896328,  4.2314855 ,  3.99647915,
       31.23958638]), 'mean_score_time': array([2.23639031, 1.3252991 , 1.73850719, 1.24541525, 1.37714493,
       1.3074249 ]), 'std_score_time': array([0.14395799, 0.13146583, 0.09331472, 0.08438314, 0.09505282,
       0.11493295]), 'param_C': masked_array(data=[0.1, 0.1, 1, 1, 10, 10],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'C': 0.1, 'kernel': 'rbf'}, {'C': 0.1, 'kernel': 'linear'}, {'C': 1, 'kernel': 'rbf'}, {'C': 1, 'kernel': 'linear'}, {'C': 10, 'kernel': 'rbf'}, {'C': 10, 'kernel': 'linear'}], 'spl

In [61]:
svc_vc_nf.score(X_test_small_nf, y_test_small_nf_m)

0.945991442800028

In [59]:
import joblib
svc_best_nf = svc_vc_nf.best_estimator_
joblib.dump(svc_best_nf, os.path.join(os.curdir,'Models', 'model_svc_survive_nf.joblib'))
svc_best_loaded_nf = joblib.load(os.path.join(os.curdir,'Models', 'model_svc_survive_nf.joblib'))

In [60]:
#svc_best_loaded.predict([X_test_small.iloc[0, :]])
svc_best_loaded_nf.predict([[i for i in range(24)]])

array([0], dtype=int64)

2. ** With fires**

In [62]:
parameters = {'C': [0.1, 1, 10], 'kernel': ['rbf', 'linear']}
from sklearn.model_selection import GridSearchCV
clf_fs_2 = SVC(gamma='auto', verbose=True)
svc_vc_fs = GridSearchCV(clf_fs_2, parameters, cv=6, refit=True)
svc_vc_fs.fit(X_train_small_fs, y_train_small_fs_m)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

GridSearchCV(cv=6, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.1, 1, 10], 'kernel': ['rbf', 'linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [63]:
print(svc_vc_fs.cv_results_)

{'mean_fit_time': array([ 8.00113336,  7.7794445 ,  7.87745432, 15.08317482, 10.09467606,
       72.6587652 ]), 'std_fit_time': array([0.39169842, 0.32830542, 0.13949482, 0.41731323, 0.5491174 ,
       5.80136382]), 'mean_score_time': array([0.78524526, 0.38637201, 0.64806481, 0.38803879, 0.54238756,
       0.47571421]), 'std_score_time': array([0.03032576, 0.01874297, 0.0480326 , 0.02326182, 0.04925825,
       0.03367164]), 'param_C': masked_array(data=[0.1, 0.1, 1, 1, 10, 10],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'C': 0.1, 'kernel': 'rbf'}, {'C': 0.1, 'kernel': 'linear'}, {'C': 1, 'kernel': 'rbf'}, {'C': 1, 'kernel': 'linear'}, {'C': 10, 'kernel': 'rbf'}, {'C': 10, 'kernel': 'linear'}], 'split0_test_sco

In [64]:
svc_vc_fs.score(X_test_small_fs, y_test_small_fs_m)

0.9381966670345436

In [65]:
import joblib
svc_best_fs = svc_vc_fs.best_estimator_
joblib.dump(svc_best_fs, os.path.join(os.curdir,'Models', 'model_svc_survive_fs.joblib'))
svc_best_loaded_fs = joblib.load(os.path.join(os.curdir,'Models', 'model_svc_survive_fs.joblib'))

In [66]:
#svc_best_loaded.predict([X_test_small.iloc[0, :]])
svc_best_loaded_fs.predict([[i for i in range(24)]])

array([0], dtype=int64)

## 6. Using models in games

1. **no fires**

**enemy frequency = 1 / 2**

The original data with no fires has a ratio of movements vs cost lives to be (5 test runs): 

|              |    1    |    2    |    3    |    4    |    5    |
|--------------|--------:|--------:|--------:|--------:|--------:|
|Lives Consumed| 2372.000| 2770.000| 2627.000| 2250.000| 1989.000|
|Moves         |12000.000|12000.000|12000.000|12000.000|12000.000|
|Ratio         |    0.198|    0.231|    0.219|    0.187|    0.166|

After training the data ratio becomes (5 test runs): 

|          |    1    |    2    |    3    |    4    |    5    |
|----------|--------:|--------:|--------:|--------:|--------:|
|Lives Consumed| 2448.000| 2002.000| 2445.000| 2263.000| 2313.000|
|Moves     |12001.000|12001.000|12001.000|12001.000|12001.000|
|Ratio     |    0.204|    0.167|    0.204|    0.189|    0.193|


The improvements are not very high, because there is a lot of topedoes and enemies in the game.

Next I will try to decrease the amount of topedoes and enemies, to see if there will be higher improvement

**enemy frequency = 1 / 8**



The original data with no fires has a ratio of movements vs cost lives to be (5 test runs): 



After training the data ratio becomes (5 test runs): 




The improvements are 

2. ** With fires**